In [1]:
import numpy as np
import pandas as pd
import cv2
import os

In [2]:
img = cv2.imread('cTDaR_t00040_0_H.jpg', cv2.IMREAD_GRAYSCALE)
rows, cols = img.shape

In [53]:
class VerticalRunLength:
    def __init__(self, x = 0, y_start = 0, y_end = 0):
        self.x = x
        self.y_start = y_start
        self.y_end = y_end
class ChainNode:
    def __init__(self, center_y, rl_info, p=0):
        self.center_y = center_y
        self.runlength_info = rl_info
        self.next = p
class Chain:
    def __init__(self, x_start, x_end, y_avg, head):
        self.x_start = x_start
        self.x_end = x_end
        self.y_avg = y_avg
        self.head = head

In [54]:
def chain_filter(chain, rows, cols): 
    if  chain.x_end-chain.x_start<=cols/40:
        return False
    else:
        return True
    
    
def merge_chain(rl_Head):
    ptr = rl_Head
    x_start = ptr.runlength_info.x
    x_end = x_start
    y_sum = ptr.runlength_info.y_end-ptr.runlength_info.y_start
    cnt = 1
    
    ptr = rl_Head.next
    while ptr!=0:
        x_end = ptr.runlength_info.x
        y_sum+=ptr.runlength_info.y_end-ptr.runlength_info.y_start
        cnt+=1
        ptr = ptr.next
    chain = Chain(x_start, x_end, y_sum/cnt, rl_Head)
    return chain
    
    
def getchain(cur_col, cur_row, img, visited):

    rl_Head = None
    last_rl = None
    last_node = None

    x = cur_col
    ys = cur_row
    ye = ys
    while ye+1<rows and img[ye+1][x]>0 and visited[ye+1][x]==0:
        ye = ye+1
    last_rl = VerticalRunLength(x, ys, ye)
    center_y = (ys+ye)/2
    last_node = ChainNode(center_y, last_rl)
    rl_Head = last_node

    for pix in range(ys, ye+1):
        visited[pix][x] = 1


    while x+1<cols:
        x = last_rl.x + 1
        ys = last_rl.y_start
        ye = last_rl.y_end + 1
        end_flag = False

        while ys<ye and img[ys][x]==0 and visited[ys][x]==0:
            ys = ys+1
        while ys<ye and img[ye-1][x]==0 and visited[ye-1][x]==0:
            ye = ye-1
        if ys==ye:
            end_flag = True
            break

        y_start = ys
        y_end = ye

        while ys<ye and img[ys][x]>0 and visited[ys][x]==0:
            ys = ys+1
        while ys<ye and img[ye-1][x]>0 and visited[ye-1][x]==0:
            ye = ye-1
        if ys!=ye:
            end_flag = True
            break

        ys = y_start
        ye = y_end - 1
        while ys-1>=0 and img[ys-1][x]>0 and visited[ys-1][x]==0:
            ys = ys-1
        while ye+1<rows and img[ye+1][x]>0 and visited[ye+1][x]==0:
            ye = ye+1

        curr_rl = VerticalRunLength(x, ys, ye)
        center_y = (ys+ye-1)/2
        # 更新last_rl last_node
        last_rl = curr_rl
        curr_node = ChainNode(center_y, curr_rl)
        last_node.next = curr_node
        last_node = curr_node
        for pix in range(ys, ye+1):
            visited[pix][x] = 1
    return rl_Head, visited

## get chain

In [55]:
chains = []
visited = np.zeros((rows, cols))
last_rl = None    # 上一游标
last_node = None  # 上一链表节点

for row in range(rows):
    for col in range(cols):
        if img[row][col]>0 and visited[row][col]==0 :
            chain_head, visited = getchain(col, row, img, visited)
            if chain_head!=None:
                chain = merge_chain(chain_head)
                if chain_filter(chain, rows, cols):
                    chains.append(chain)

In [56]:
len(chains)

225

## match